# VGG16
- 가중치 초기화
- 데이터셋으로 직접 학습
- 입력 데이터 사이즈 (128,128,3)
- 옵티마이저 adam, 학습률 default(0.001)
- train 데이터 배치사이즈 100
- 과적합 방지를 위해 earlystopping 적용

In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/안봐도보이조_project

/content/drive/MyDrive/안봐도보이조_project


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,Flatten,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model

In [ ]:
# 데이터셋 준비
train_datagen = ImageDataGenerator(rescale=1./255,         # 픽셀 값을 0~1 범위로 변환(scaling)
                                   rotation_range=40,      # 40도까지 회전
                                   width_shift_range=0.2,  # 20%까지 좌우 이동
                                   height_shift_range=0.2, # 20%까지 상하 이동
                                   shear_range=0.2,        # 20%까지 기울임
                                   zoom_range=0.2,         # 20%까지 확대/축소
                                   horizontal_flip=True)   # 좌우 뒤집기
    
train_generator = train_datagen.flow_from_directory('./train_10', # 데이터가 존재하는 디렉토리
                                                    batch_size=100, # 배치사이즈(변경가능)
                                                    class_mode='categorical', # 다중 분류이므로 categorical
                                                    target_size=(128, 128)) # 모델에 입력해줄 데이터 사이즈(변경가능)

valid_datagen = ImageDataGenerator(rescale=1./255) # 스케일링만 진행
valid_generator = valid_datagen.flow_from_directory('./valid_10',
                                                    batch_size=20,      
                                                    class_mode='categorical', 
                                                    target_size=(128, 128))

test_datagen = ImageDataGenerator(rescale=1./255) # 스케일링만 진행
test_generator = test_datagen.flow_from_directory('./test_10',
                                                  batch_size=10,      
                                                  class_mode='categorical', 
                                                  target_size=(128, 128),shuffle=False)

Found 10529 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 500 images belonging to 10 classes.


In [ ]:
# 클래스 확인
print(train_generator.class_indices)

{'갈아만든배': 0, '레쓰비': 1, '마운틴듀': 2, '밀키스': 3, '스프라이트': 4, '칠성사이다': 5, '코카콜라': 6, '트로피카나망고': 7, '펩시콜라': 8, '환타오렌지': 9}


In [ ]:
# 모델 구성 및 확인
model = VGG16(weights=None, classes=10, input_shape=(128,128,3))
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [ ]:
# 모델 컴파일
model.compile(optimizer = Adam(), # 최적화함수 adam사용
              loss = 'categorical_crossentropy', # 손실함수 범주형 교차엔트로피 사용
              metrics = ['accuracy']) # 지표는 정확도


In [8]:
# 모델 학습
earlystopping = EarlyStopping(monitor='val_loss', patience=5) # 5번동안 성능개선이 없으면 학습 조기종료

history = model.fit(train_generator,
                    steps_per_epoch=100,  
                    epochs=100, 
                    validation_data=valid_generator, 
                    validation_steps=30, callbacks=[earlystopping]) 
              
# 학습률을 default로 사용했을 때 아무리 학습을 진행 시켜도 정확도가 0.1에서 개선되지 않음

Epoch 1/100
100/100 [==============================] - 2900s 29s/step - loss: 2.3074 - accuracy: 0.1117 - val_loss: 2.3053 - val_accuracy: 0.1017
Epoch 2/100
100/100 [==============================] - 252s 3s/step - loss: 2.2994 - accuracy: 0.1148 - val_loss: 2.3100 - val_accuracy: 0.0867
Epoch 3/100
100/100 [==============================] - 82s 823ms/step - loss: 2.2990 - accuracy: 0.1166 - val_loss: 2.3041 - val_accuracy: 0.1133
Epoch 4/100
100/100 [==============================] - 64s 640ms/step - loss: 2.2992 - accuracy: 0.1175 - val_loss: 2.3123 - val_accuracy: 0.0933
Epoch 5/100
100/100 [==============================] - 55s 543ms/step - loss: 2.2987 - accuracy: 0.1159 - val_loss: 2.3043 - val_accuracy: 0.1033
Epoch 6/100
100/100 [==============================] - 53s 531ms/step - loss: 2.2994 - accuracy: 0.1166 - val_loss: 2.3053 - val_accuracy: 0.1000
Epoch 7/100
100/100 [==============================] - 53s 527ms/step - loss: 2.2993 - accuracy: 0.1172 - val_loss: 2.3079 - v

In [ ]:
# 모델 평가 미진행